In [25]:
import pandas as pd
import feature_engineering
import quadratic_weighted_kappa

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

%reload_ext autoreload
%autoreload 2

In [13]:
data = pd.read_csv('data_minus_images/train.csv')

In [14]:
pet_df = feature_engineering.add_everything(data)

In [15]:
pet_df.head()

,PetID,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,...,score,language,BreedName,BreedGroup,BreedGroupID,purebred,desc_len,start_cap,num_colors,has_name
0,86e1089a3,2,Nibble,3,299,0,1,1,7,0,...,0.3,en,Tabby,CAT,-1,1,359,1,2,1
1,1bf1f95ef,2,Mimi,12,299,0,2,3,6,7,...,0.0,None,Tabby,CAT,-1,1,40,1,3,1
2,987baa1c7,2,Tabby Male,48,299,0,1,1,2,7,...,0.3,en,Tabby,CAT,-1,1,707,1,3,1
3,5c546c236,2,Manja,4,299,0,2,1,2,0,...,0.4,en,Tabby,CAT,-1,1,130,1,2,1
4,c2346525a,2,Cham,8,299,0,1,3,0,0,...,0.5,en,Tabby,CAT,-1,1,99,1,1,1


In [17]:
X = pet_df.drop(columns=['Name','RescuerID','Description','PetID','AdoptionSpeed', 'BreedName', 'BreedGroupID'])
y = pet_df['AdoptionSpeed'].astype('str')

In [18]:
cat_features = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'State', 'language', 'BreedGroup']

In [19]:
for feat in cat_features:
    dummies = pd.get_dummies(X[feat], prefix=feat)
    dummies.drop(dummies.columns[-1], axis=1, inplace=True)
    X = X.drop(feat, axis=1).merge(dummies, left_index=True, right_index=True)

In [20]:
y_one = pd.get_dummies(y)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [112]:
nn = Sequential()
nn.add(Dense(64, activation='relu', input_dim=(X_train.shape[1])))
nn.add(Dropout(0.5))
nn.add(Dense(128, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(128, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(128, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(128, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(32, activation='relu'))
nn.add(Dropout(0.5))
nn.add(Dense(1,activation='linear'))

In [69]:
import keras.backend as K

def kappa(y_true, y_pred):
    kappa = quadratic_weighted_kappa.quadratic_weighted_kappa(y_true, y_pred)
    return K.sum(kappa)

In [113]:
nn.compile(loss=keras.losses.mean_absolute_error, optimizer=keras.optimizers.Adam(), metrics=['mae'])

trained = nn.fit(X_train, y_train, batch_size=64,epochs=40,verbose=1,validation_data=(X_test, y_test))

nn.evaluate(X_test, y_test)

Train on 11990 samples, validate on 2998 samples
Epoch 1/40
11990/11990 [==============================] - 2s 167us/step - loss: 3.9778 - mean_absolute_error: 3.9778 - val_loss: 2.1627 - val_mean_absolute_error: 2.1627
Epoch 2/40
11990/11990 [==============================] - 1s 80us/step - loss: 1.7983 - mean_absolute_error: 1.7983 - val_loss: 1.7074 - val_mean_absolute_error: 1.7074
Epoch 3/40
11990/11990 [==============================] - 1s 79us/step - loss: 1.4870 - mean_absolute_error: 1.4870 - val_loss: 1.1346 - val_mean_absolute_error: 1.1346
Epoch 4/40
11990/11990 [==============================] - 1s 79us/step - loss: 1.2273 - mean_absolute_error: 1.2273 - val_loss: 1.0323 - val_mean_absolute_error: 1.0323
Epoch 5/40
11990/11990 [==============================] - 1s 82us/step - loss: 1.1405 - mean_absolute_error: 1.1405 - val_loss: 1.0315 - val_mean_absolute_error: 1.0315
Epoch 6/40
11990/11990 [==============================] - 1s 82us/step - loss: 1.0992 - mean_absolute_err

[0.9196095448720447, 0.9196095448720447]

In [114]:
y_predict = nn.predict(X_test)

In [115]:
y_train.astype('int').mean()

2.5155963302752293

In [116]:
y_predict

array([[2.0234296],
       [2.0020607],
       [1.9999936],
       ...,
       [2.9937322],
       [2.9937322],
       [2.003214 ]], dtype=float32)

In [117]:
y_predict = y_predict.reshape((1,-1))

In [118]:
y_predict = y_predict.round()[0]

In [119]:
y_predict

array([2., 2., 2., ..., 3., 3., 2.], dtype=float32)

In [120]:
quadratic_weighted_kappa.quadratic_weighted_kappa(y_predict, y_test)

0.18263729927821437

In [35]:
y_test_class = y_test.idxmax(axis=1)

In [57]:
y_test.head()

,0,1,2,3,4
5992,0,0,0,1,0
9981,0,0,0,1,0
12717,0,0,1,0,0
6518,0,0,1,0,0
8883,0,0,0,1,0


In [64]:
y_train.head()

,0,1,2,3,4
14472,1,0,0,0,0
820,0,0,0,1,0
1643,0,0,0,1,0
7530,0,1,0,0,0
11851,0,0,1,0,0


In [53]:
y_test_class.head()

5992     3
9981     3
12717    2
6518     2
8883     3
dtype: object

In [62]:
print(y_predict[:20])

[2 4 2 4 2 2 4 2 4 4 4 4 4 4 4 4 2 4 2 4]


In [63]:
pet_df['AdoptionSpeed'].head(20)

0     2
1     4
2     2
3     4
4     2
5     2
6     2
7     4
8     3
9     4
10    2
11    2
12    2
13    4
14    4
15    2
16    2
17    1
18    2
19    3
Name: AdoptionSpeed, dtype: int64

In [55]:
y.head()

0    2
1    4
2    2
3    4
4    2
Name: AdoptionSpeed, dtype: object

In [56]:
y_one.head()

,0,1,2,3,4
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,1,0,0
3,0,0,0,0,1
4,0,0,1,0,0
